In [ ]:
# import
import csv
import os
import random
import string

import cv2
import matplotlib.pyplot as plt
import numpy as np  # linear algebra
import torch
import torch.nn as nn
import torch.nn.functional as Fun
import torchvision.transforms as T
from torch.utils.data import DataLoader, Dataset
from torchvision import models
from tqdm import trange

TEST_PATH = "/content/drive/MyDrive/data/test"
# TRAIN_PATH = "/kaggle/input/captcha-hacker/train"
# TEST_PATH  = "/kaggle/input/captcha-hacker/test"

class_index = dict(zip(string.ascii_lowercase+string.digits, range(36)))
class_index_rev = dict(zip(range(36), string.ascii_lowercase+string.digits))
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = 'cpu'
print(f"current using device: {device}")

In [ ]:
# Architecture of my model
class Mymodel1(nn.Module):
    def __init__(self):
        super(Mymodel1, self).__init__()

        # self.model = models.resnet18(pretrained=True)
        self.model = models.resnet18(weights='ResNet50_Weights.DEFAULT')

    # for param in self.model.parameters():
      # param.requires_grad = False

        self.model.fc = nn.Linear(512, 10)
    def forward(self, x):
        logits = self.model(x)
        return logits
class Mymodel2(nn.Module):
    def __init__(self):
        super(Mymodel2, self).__init__()

        # self.model = models.resnet18(pretrained=True)
        self.model = models.resnet18(weights='ResNet50_Weights.DEFAULT')

    # for param in self.model.parameters():
      # param.requires_grad = False
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Identity()
        self.fc1 = nn.Linear(num_ftrs, 36)
        self.fc2 = nn.Linear(num_ftrs, 36)
    def forward(self, x):
        x = self.model(x)
        out1 = self.fc1(x)
        out2 = self.fc2(x)
        return out1, out2        
class Mymodel3(nn.Module):
    def __init__(self):
        super(Mymodel3, self).__init__()

        # self.model = models.resnet18(pretrained=True)
        self.model = models.resnet18(weights='ResNet50_Weights.DEFAULT')

    # for param in self.model.parameters():
      # param.requires_grad = False

        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Identity()
        self.fc1 = nn.Linear(num_ftrs, 36)
        self.fc2 = nn.Linear(num_ftrs, 36)
        self.fc3 = nn.Linear(num_ftrs, 36)
        self.fc4 = nn.Linear(num_ftrs, 36)
    def forward(self, x):
        x = self.model(x)
        logits1 = self.fc1(x)
        logits2 = self.fc2(x)
        logits3 = self.fc3(x)
        logits4 = self.fc4(x)
        return logits1, logits2, logits3, logits4

In [ ]:
# Download the weight of my model
FILE1 = "Task1_weights.pt"
FILE2 = "Task2_weights.pt"
FILE3 = "Task3_weights.pt"
root = "/content/drive/MyDrive/HW5_model_weights" # fill the path you store my weight
# Load weight
model1 = Mymodel1()
model1.load_state_dict(torch.load(f"{root}/{FILE1}"))
model2 = Mymodel2()
model2.load_state_dict(torch.load(f"{root}/{FILE2}"))
model3 = Mymodel3()
model3.load_state_dict(torch.load(f"{root}/{FILE3}"))

# Should output a warning (pretrained is deprecated..)

In [ ]:
def read_data(rows, prefix, root):
    filenames = np.array([sample for sample in rows if sample[0].startswith(prefix)])
    imgs = filenames[:, 0]
    label = filenames[:, 1]
    images = np.zeros(shape=(len(filenames), 64, 64, 3), dtype=np.uint8)
    labels = np.zeros(shape=(len(filenames), len(label[0])), dtype=np.int32)
    for i in trange(len(filenames)):
        curr_img = cv2.imread(f"{root}/{imgs[i]}")
        curr_img = cv2.resize(curr_img,  (64, 64))
        images[i] = curr_img
        if prefix == "task1":
            curr_label = [class_index[x]-26 for x in label[i]]
        else:
            curr_label = [class_index[x] for x in label[i]]
        labels[i] = np.array(curr_label)
    return np.array(imgs), np.array(images), np.array(labels)
class ImgDataset(Dataset):
    def __init__(self, x_data, y_data, root, filenames, return_filename=False, prefix="task1"):
        x_data = x_data.astype('float32')
        self.x_data = torch.from_numpy(x_data).permute(0, 3, 1, 2) # (N, 1, 64, 64)
        self.y_data = torch.from_numpy(y_data) 
        self.filenames = filenames
        self.return_filename = return_filename

    def __getitem__(self, index):
        if self.return_filename:
            return self.x_data[index], self.filenames[index] 
        else:
            return self.x_data[index], self.y_data[index]

    def __len__(self):
        return len(self.x_data)

In [ ]:
test_data = []
with open(f'{TEST_PATH}/../sample_submission.csv', newline='') as csvfile:
    for row in csv.reader(csvfile, delimiter=','):
        test_data.append(row)

csv_writer = csv.writer(open('submission.csv', 'w', newline=''))
csv_writer.writerow(["filename", "label"])

test_path, x_test, y_test = read_data(rows=test_data, prefix="task1", root=TEST_PATH)
test_ds1 = ImgDataset(x_data=x_test, y_data=y_test, root=TEST_PATH, filenames=test_path, return_filename=True, prefix="task1")
test_dl1 = DataLoader(test_ds1, batch_size=500, num_workers=2, drop_last=False, shuffle=False)

test_path, x_test, y_test = read_data(rows=test_data, prefix="task2", root=TEST_PATH)
test_ds2 = ImgDataset(x_data=x_test, y_data=y_test, root=TEST_PATH, filenames=test_path, return_filename=True, prefix="task2")
test_dl2 = DataLoader(test_ds2, batch_size=500, num_workers=2, drop_last=False, shuffle=False)

test_path, x_test, y_test = read_data(rows=test_data, prefix="task3", root=TEST_PATH)
test_ds3 = ImgDataset(x_data=x_test, y_data=y_test, root=TEST_PATH, filenames=test_path, return_filename=True, prefix="task3")
test_dl3 = DataLoader(test_ds3, batch_size=500, num_workers=2, drop_last=False, shuffle=False)


In [ ]:
csv_writer = csv.writer(open('submission.csv', 'w', newline=''))
csv_writer.writerow(["filename", "label"])

model1 = model1.to(device)
model1.eval()
for image, filenames in test_dl1:
    image = image.to(device)
    
    pred = model1(image)
    pred = torch.argmax(pred, dim=1)
    
    for i in range(len(filenames)):
        curr_string = class_index_rev[pred[i].item()+26]
#         print(f"Write: {filenames[i]},{curr_string}")
        csv_writer.writerow([filenames[i], str(pred[i].item())])
print("Task1: done")


model2 = model2.to(device)
model2.eval()
for image, filenames in test_dl2:
    image = image.to(device)
    
    output = model2(image)
    pred1 = torch.argmax(output[0], dim=1)
    pred2 = torch.argmax(output[1], dim=1)
    
    for i in range(len(filenames)):
        curr_string = class_index_rev[pred1[i].item()]+class_index_rev[pred2[i].item()]
#         print(f"Write: {filenames[i]},{curr_string}")
        csv_writer.writerow([filenames[i], curr_string])
print("Task2: done")

model3 = model3.to(device)
model3.eval()
for image, filenames in test_dl3:
    image = image.to(device)
    
    output = model3(image)
    pred1 = torch.argmax(output[0], dim=1)
    pred2 = torch.argmax(output[1], dim=1)
    pred3 = torch.argmax(output[2], dim=1)
    pred4 = torch.argmax(output[3], dim=1)
    for i in range(len(filenames)):
        curr_string = class_index_rev[pred1[i].item()]+class_index_rev[pred2[i].item()]+class_index_rev[pred3[i].item()]+class_index_rev[pred4[i].item()]
#         print(f"Write: {filenames[i]},{curr_string}")
        csv_writer.writerow([filenames[i], curr_string])
print("Task3: done")
